## Using __call__ rather that forward

In [1]:
%matplotlib inline

import numpy as np
import torch
import torch.optim as optim

torch.set_printoptions(edgeitems=2,linewidth=75)

In [2]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape

torch.Size([11, 1])

In [3]:
t_c

tensor([[ 0.5000],
        [14.0000],
        [15.0000],
        [28.0000],
        [11.0000],
        [ 8.0000],
        [ 3.0000],
        [-4.0000],
        [ 6.0000],
        [13.0000],
        [21.0000]])

In [4]:
n_samples = t_u.shape[0]
n_val = int(0.2*n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([1, 3, 0, 4, 7, 2, 6, 8, 5]), tensor([ 9, 10]))

In [5]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

In [6]:
import torch.nn as nn
linear_model = nn.Linear(1,1)
linear_model(t_un_val)

tensor([[3.2379],
        [3.6999]], grad_fn=<AddmmBackward>)

In [7]:
linear_model.weight

Parameter containing:
tensor([[0.5774]], requires_grad=True)

In [8]:
linear_model.bias

Parameter containing:
tensor([-0.2496], requires_grad=True)

In [9]:
x = torch.ones(1)

In [10]:
linear_model(x)

tensor([0.3278], grad_fn=<AddBackward0>)

In [11]:
x = torch.ones(10,1)

In [12]:
linear_model(x)

tensor([[0.3278],
        [0.3278],
        [0.3278],
        [0.3278],
        [0.3278],
        [0.3278],
        [0.3278],
        [0.3278],
        [0.3278],
        [0.3278]], grad_fn=<AddmmBackward>)

In [13]:
linear_model = nn.Linear(1,1)
optimizer = optim.SGD(
linear_model.parameters(),
lr=1e-2)

In [14]:
linear_model.parameters()

<generator object Module.parameters at 0x000001E734B3E660>

In [15]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[0.3612]], requires_grad=True),
 Parameter containing:
 tensor([0.1659], requires_grad=True)]

In [16]:
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) 
t_u = torch.tensor(t_u).unsqueeze(1)

In [17]:
t_u.shape

torch.Size([11, 1])

In [18]:
linear_model = nn.Linear(1,1)
optimizer = optim.SGD(
linear_model.parameters(),
lr=1e-2)

In [19]:
linear_model.parameters()

<generator object Module.parameters at 0x000001E734B3E350>

In [20]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[0.1534]], requires_grad=True),
 Parameter containing:
 tensor([-0.8399], requires_grad=True)]

In [21]:
def training_loop(n_epochs,optimizer,model,loss_fn,t_u_train,t_u_val,t_c_train,t_c_val):
    for epoch in range(1,n_epochs+1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train,t_c_train)
        
        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val,t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        
        if epoch == 1 or epoch% 1000 == 0:
            print(f"Epoch {epoch},Training loss {loss_train.item():.4f},",
                 f"Valication loss {loss_val.item():.4f}")

In [22]:
linear_model = nn.Linear(1,1)
optimizer = optim.SGD(linear_model.parameters(),lr=1e-2)

In [23]:
training_loop(
n_epochs=3000,
optimizer=optimizer,
model=linear_model,
loss_fn=nn.MSELoss(),
t_u_train = t_un_train,
t_u_val = t_un_val,
t_c_train=t_c_train,
t_c_val = t_c_val)

Epoch 1,Training loss 157.2794, Valication loss 297.4794
Epoch 1000,Training loss 3.4272, Valication loss 4.2343
Epoch 2000,Training loss 2.8073, Valication loss 3.6189
Epoch 3000,Training loss 2.7946, Valication loss 3.5613


In [24]:
seq_model  = nn.Sequential(
nn.Linear(1,13),
nn.Tanh(),
nn.Linear(13,1))

In [25]:
seq_model

Sequential(
  (0): Linear(in_features=1, out_features=13, bias=True)
  (1): Tanh()
  (2): Linear(in_features=13, out_features=1, bias=True)
)

In [26]:
[param.shape for param in seq_model.parameters()]

[torch.Size([13, 1]), torch.Size([13]), torch.Size([1, 13]), torch.Size([1])]

In [27]:
for name,param in seq_model.named_parameters():
    print(name,param.shape)

0.weight torch.Size([13, 1])
0.bias torch.Size([13])
2.weight torch.Size([1, 13])
2.bias torch.Size([1])


In [28]:
from collections import OrderedDict
seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear',nn.Linear(1,8)),
    ('hidden_activation',nn.Tanh()),
    ('output_linear',nn.Linear(8,1))
]))

In [29]:
seq_model

Sequential(
  (hidden_linear): Linear(in_features=1, out_features=8, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=8, out_features=1, bias=True)
)

In [30]:
for name, param in seq_model.named_parameters():
    print(name,param.shape)

hidden_linear.weight torch.Size([8, 1])
hidden_linear.bias torch.Size([8])
output_linear.weight torch.Size([1, 8])
output_linear.bias torch.Size([1])


In [31]:
seq_model.output_linear.bias

Parameter containing:
tensor([-0.3273], requires_grad=True)

In [32]:
optimizer = optim.SGD(seq_model.parameters(),lr=1e-3)

In [33]:
training_loop(
n_epochs=5000,
optimizer=optimizer,
model=seq_model,
loss_fn=nn.MSELoss(),
t_u_train= t_un_train,
t_u_val = t_un_val,
t_c_train=t_c_train,
t_c_val=t_c_val)

Epoch 1,Training loss 159.7114, Valication loss 302.1733
Epoch 1000,Training loss 6.0721, Valication loss 5.1753
Epoch 2000,Training loss 2.9246, Valication loss 7.4086
Epoch 3000,Training loss 1.6993, Valication loss 4.0574
Epoch 4000,Training loss 1.6040, Valication loss 4.1869
Epoch 5000,Training loss 1.5413, Valication loss 4.4667


In [34]:
# from matplotlib import pyplot as plt
# t_range=torch.arange(20.,90.).unsqueeze(1)
# fig = plt.figure(dpi=600)

# plt.xlabel("fahrenheit")
# plt.ylabel("Celsius")
# plt.plot(t_u.numpy(),t_c.numpy(),'o')
# plt.plot(t_range.numpy(),seq_model(0.1*t_range).detach().numpy(),'c-')
# plt.plot(t_u.numpy(),seq_model(0.1*t_u).detach().numpy(),'kx')